In [2]:
using Revise

In [3]:
using ITensors
using Test

In [4]:
import CTMRG

[ Info: Precompiling CTMRG [c050b7f9-16a0-4967-b3f3-566d5b3f75cc] (cache misses: include_dependency fsize change (2))


In [5]:
function ising_transfer_element_1D(i1::Int, i2::Int,β::Float64,J::Float64)
    s1 = 2*i1 - 1
    s2 = 2*i2 - 1
    expvalue = exp(β*J*s1*s2)
    return expvalue
end

ising_transfer_element_1D (generic function with 1 method)

In [6]:
@testset "ising_transfer_element_1D" begin
    β = 1.0
    J = 1.0
    @test ising_transfer_element_1D(0, 0, β, J) ≈ exp(β*J)
    @test ising_transfer_element_1D(0, 1, β, J) ≈ exp(-β*J)
    @test ising_transfer_element_1D(1, 0, β, J) ≈ exp(-β*J)     
    @test ising_transfer_element_1D(1, 1, β, J) ≈ exp(β*J)    
end
;

Test Summary:             | Pass  Total  Time
ising_transfer_element_1D |    4      4  0.2s


In [7]:
#テンソル行列に値をセットする
s1 = Index(2, "s1,m=1")
s2 = Index(2, "s2,n=1")
T = ITensor(s1, s2)
for i in 0:1, j in 0:1
    T[s1=>i+1, s2=>j+1] = ising_transfer_element_1D(i, j, 1.0, 1.0)
end
T
;


In [8]:
#1Dイジングの転送行列を作る関数
function ising_transfer_matrix_1D(β::Float64, J::Float64,s1::Index, s2::Index)
    T = ITensor(s1, s2)
    for i in 0:1, j in 0:1
        T[s1=>i+1, s2=>j+1] = ising_transfer_element_1D(i, j, β, J)
    end
    return T
end

ising_transfer_matrix_1D (generic function with 1 method)

In [ ]:
@testset "ising_transfer_matrix_1Dのテスト" begin
    β = 1.0
    J = 1.0
    s1 = Index(2, "s1,m=1")
    s2 = Index(2, "s2,n=1")
    T = ising_transfer_matrix_1D(β,J,s1,s2)
    @test T[s1=>1, s2=>1] ≈ exp(β*J)
end

Test Summary:                 | Pass  Total  Time
ising_transfer_element_1Dのテスト |    1      1  0.0s


Test.DefaultTestSet("ising_transfer_element_1Dのテスト", Any[], 1, false, false, true, 1.754278102838814e9, 1.754278102886097e9, false, "In[9]")

In [10]:
#境界条件を表す関数
function freeend(ind1::Index)
    D = ITensor(ind1)
    D[ind1 => 1] = 1.0
    D[ind1 => 2] = 1.0
    return D
end
function fixend(ind1::Index)
    D = ITensor(ind1)
    D[ind1 => 1] = 1.0
    D[ind1 => 2] = 0.0
    return D
end 

fixend (generic function with 1 method)

In [11]:
#縮約の練習
s1 = Index(2, "s1,n=1")
s2 = Index(2, "s2,n=2")
R = fixend(s1)
T = ising_transfer_matrix_1D(1.0, 1.0, s1, s2)
L = fixend(s2)
RT = R * T
LRT = RT * L
;

In [12]:
#分配関数の計算
N = 4
β = 1.0
J = 1.0
s_list = Vector{Index}(undef, N) 
T_list = Vector{ITensor}(undef, N-1)          # 転送行列を格納する配列
boundary = Vector{ITensor}(undef, 2)
for i in 1:N
    s_list[i] = Index(2, "s, n=$i")
end
for i in 1:N
    if i == 1
        boundary[1] = fixend(s_list[1])
        T_list[i] = ising_transfer_matrix_1D(β, J, s_list[i], s_list[i+1])
    elseif i == N
        boundary[2] = fixend(s_list[N])
    else 
        T_list[i] = ising_transfer_matrix_1D(β, J, s_list[i], s_list[i+1])
    end
end
#=
for i in 0:N-1
    s_list[i+1] = Index(2, "s, n=$(i+1)")
end
#@show s_list
for i in 0:1
    if i == 0
        boundary[i+1] = freeend(s_list[1])
    else
        boundary[i+1] = freeend(s_list[N])
    end
end
for i in 0:N-2
    T_list[i+1] = ising_transfer_matrix_1D(1.0, 1.0, s_list[i+1], s_list[i+2])
end
=#
;


In [13]:
#1次元イジング模型の分配関数を計算する関数
function partition_function_1D(N::Int, β::Float64, J::Float64)
    s_list = Vector{Index}(undef, N) 
    T_list = Vector{ITensor}(undef, N-1)          # 転送行列を格納する配列
    boundary = Vector{ITensor}(undef, 2)
    for i in 1:N
        s_list[i] = Index(2, "s, n=$i")
    end
    for i in 1:N
        if i == 1
            boundary[1] = fixend(s_list[1])
            T_list[i] = ising_transfer_matrix_1D(β, J, s_list[i], s_list[i+1])
        elseif i == N
            boundary[2] = fixend(s_list[N])
        else 
            T_list[i] = ising_transfer_matrix_1D(β, J, s_list[i], s_list[i+1])
        end
    end
    return s_list, T_list, boundary
end
#分配関数の縮約をとる関数
function partition_function_contraction(N::Int)
    result = ITensor(true)
    s_list, T_list, boundary = partition_function_1D(N, β, J)
    L = boundary[1]
    R = boundary[2]
    for T in T_list
        result = result * T   # 順に縮約
    end
    return only(L * result * R) #順番は任意
end

partition_function_contraction (generic function with 1 method)

In [14]:
s_list, T_list, boundary = partition_function_1D(N, 1.0, 1.0)
@show result = ITensor(true)
L = boundary[1]
R = boundary[2]
for T in T_list
    result = result * T   # 順に縮約
end
@show Z = only(L * result * R)
@show ref_Z = 0.5 * (2*cosh(β*J))^(N-1) + 0.5 * (2*sinh(β*J))^(N-1)
;

result = ITensor(true) = ITensor ord=0
NDTensors.Dense{Bool, Vector{Bool}}
 0-dimensional
true
Z = only(L * result * R) = 21.189175246701993
ref_Z = 0.5 * (2 * cosh(β * J)) ^ (N - 1) + 0.5 * (2 * sinh(β * J)) ^ (N - 1) = 21.189175246701993


$$一次元イジング模型の期待値を求める

In [27]:
function ising_spin_expectation_1D(N::Int, i1::Int, β::Float64, J::Float64)
    s_list = Vector{Index}(undef, N) 
    T_list = Vector{ITensor}(undef, N-1)          # 転送行列を格納する配列
    boundary = Vector{ITensor}(undef, 2)
    for i in 1:N
        s_list[i] = Index(2, "s, n=$i")
    end
    s_target = Index(2, "s_target")
    spin_tensor = CTMRG.spin_tensor(s_list[i1], s_target)
    for i in 1:N
        if i == 1
            boundary[1] = fixend(s_list[1])
            T_list[i] = ising_transfer_matrix_1D(β, J, s_list[i], s_list[i+1])
        elseif i == i1
            #T_list[i] = ising_transfer_matrix_1D(β, J, s_list[i], s_target)
            T_list[i] = ising_transfer_matrix_1D(β, J, s_target, s_list[i+1])
        elseif i == N
            boundary[2] = fixend(s_list[N])
        else 
            T_list[i] = ising_transfer_matrix_1D(β, J, s_list[i], s_list[i+1])
        end
    end
    return s_list, T_list, boundary, spin_tensor
end
    


ising_spin_expectation_1D (generic function with 1 method)

In [38]:
N = 4
β = 1.0
J = 1.0
s_list, T_list, boundary, spintensor = ising_spin_expectation_1D(N, 3, β, J)
result = ITensor(true)
for T_list in T_list
    result *=  T_list   # 順に縮約
end
result *= spintensor
result *= boundary[1]
result *= boundary[2]
@show only(result) / (0.5 * (2*cosh(β*J))^(N-1) + 0.5 * (2*sinh(β*J))^(N-1))
@show ref_s = tanh(β*J)

only(result) / (0.5 * (2 * cosh(β * J)) ^ (N - 1) + 0.5 * (2 * sinh(β * J)) ^ (N - 1)) = 0.9305533251033541
ref_s = tanh(β * J) = 0.7615941559557649


0.7615941559557649